# PonCenter dealer assistant PoC

- Abdullah
- Bjorn
- Maud
- Norick

## Dataset diagram
<img height="400px" src="https://dev-icp.s3.eu-central-1.amazonaws.com/dataengineering/database_diagram.png">

In [ ]:
# Packages installeren
!pip install pandas
!pip install transformers
!pip install spacy
!pip install torch
!spacy download nl_core_news_sm
!pip install sentence-transformers faiss-cpu

# Imports
from transformers import pipeline
import pandas as pd
import spacy
import torch

# Configuratie
nlp = spacy.load('nl_core_news_sm')
pd.set_option('display.max_colwidth', None)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 35.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('nl_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 101.5 MB/s eta 0:00:00


In [ ]:
# Inladen van datasets vanuit AWS S3 bucket

_S3Path = 'https://dev-icp.s3.eu-central-1.amazonaws.com/dataengineering/'

werkorders_df = pd.read_csv(_S3Path + 'workorders.csv')
berichten_df = pd.read_csv(_S3Path + 'remarks.csv')
koppeltabel_df = pd.read_csv(_S3Path + 'join.csv')
voertuigen_df = pd.read_csv(_S3Path + 'vehicles.csv')
# Overkoepelende tabel van onderdelen en arbeid regels
taken_df = pd.read_csv(_S3Path + 'works.csv') # kan je beter zien als pakketten (includeerd arbeid en onderdelen later in het process)
triplets_df = pd.read_csv(_S3Path + 'triplets.csv') # 300 aan triple pairs (https://en.wikipedia.org/wiki/Triplet_loss)

/tmp/ipython-input-3965339855.py:5: DtypeWarning: Columns (7,12,50,58) have mixed types. Specify dtype option on import or set low_memory=False.
  werkorders_df = pd.read_csv(_S3Path + 'workorders.csv')
/tmp/ipython-input-3965339855.py:6: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  berichten_df = pd.read_csv(_S3Path + 'remarks.csv')
/tmp/ipython-input-3965339855.py:8: DtypeWarning: Columns (9,54,66,70) have mixed types. Specify dtype option on import or set low_memory=False.
  voertuigen_df = pd.read_csv(_S3Path + 'vehicles.csv')


## Kort overzicht analyze op de datasets

In [ ]:
werkorders_df.head()

,id,car_vehicle_id,establishment,workshop,date_appointment,time_appointment,needs_follow_up_appointment,repeat_repair_deadline_date_time,follow_up_appointment_deadline_date_time,is_repeat_visit,...,remarks_workshop,remarks_warehouse,remarks_damage,total_labor,total_labor_finished,total_labor_need_approval,last_checkin_failure,created_by_sync,created,modified
0,745185,198965,Pon Center Amersfoort Volkswagen,Werkplaats,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,NaN,2024-10-15 11:51:53,2024-10-15 02:00:00,2024-11-06 00:26:50
1,534401,199008,Pon Center Amersfoort Skoda,Werkplaats,2023-01-10,12:15:00,0.0,NaN,NaN,NaN,...,0,0,0,0,0,0,NaN,NaN,2023-01-10 00:00:00,2023-07-09 06:37:47
2,176120,199020,Pon Center Amersfoort Skoda,Werkplaats,2020-08-14,13:15:00,0.0,NaN,NaN,NaN,...,0,0,0,0,0,0,NaN,NaN,2020-08-04 00:00:00,2020-08-04 00:00:00
3,263181,199020,Pon Dealer Amersfoort Seat,Werkplaats,2020-10-06,08:00:00,0.0,NaN,NaN,NaN,...,0,0,0,0,0,0,NaN,NaN,2020-10-01 00:00:00,2020-10-01 00:00:00
4,271416,199020,Pon Dealer Amersfoort Seat,Werkplaats,2020-08-13,08:00:00,0.0,NaN,NaN,NaN,...,0,0,0,0,0,0,NaN,NaN,2020-08-03 00:00:00,2020-08-03 00:00:00


In [ ]:
berichten_df.head()

,id,workorder_id,remark_type,crm_partner_id,department,to_crm_partner_id,remarks,read,priority,remark_status,...,in_progress_datetime,postpone_datetime,postpone_reason,remark_parent_id,crm_partner_completed_id,completed_datetime,source,source_id,created,modified
0,104233,413943,diagnosis,420380,front_desk,50.0,NaN,NaN,action-required,handled,...,NaN,NaN,NaN,NaN,319048.0,2022-02-02 21:52:00,NaN,NaN,2022-02-02 20:03:27,2022-02-02 21:52:00
1,104234,413944,labor_signed,7,chief_workshop,50.0,De status is gewijzigd van 1 arbeidsregel(s) via de werkzaamheden tab,NaN,action-required,handled,...,NaN,NaN,NaN,NaN,319048.0,2022-02-02 20:44:12,NaN,NaN,2022-02-02 20:40:37,2022-02-02 20:44:12
2,104235,413944,diagnosis,319048,warehouse,50.0,test,NaN,action-required,handled,...,2022-02-02 20:43:49,NaN,NaN,NaN,319048.0,2022-02-02 20:43:52,NaN,NaN,2022-02-02 20:43:36,2022-02-02 20:43:52
3,104236,413944,diagnosis,319048,front_desk,50.0,test,NaN,action-required,handled,...,NaN,NaN,NaN,NaN,319048.0,2022-02-02 20:44:04,NaN,NaN,2022-02-02 20:43:52,2022-02-02 20:44:04
4,104237,413944,labor_signed,7,chief_workshop,50.0,De status is gewijzigd van 1 arbeidsregel(s) via de werkzaamheden tab,NaN,action-required,handled,...,NaN,NaN,NaN,NaN,842371.0,2022-02-02 21:07:38,NaN,NaN,2022-02-02 20:45:57,2022-02-02 21:07:38


In [ ]:
print('Unique departments: ', berichten_df['department'].unique())
print('Unique types: ', berichten_df['remark_type'].unique())

Unique departments:  ['front_desk' 'chief_workshop' 'warehouse' 'warranty' 'workshop'
 'orderpicker' 'kcc' 'backoffice' 'telephone_operators']
Unique types:  ['diagnosis' 'labor_signed' 'warranty' 'check_delivery'
 'customer_feedback' 'other' 'make_appointment' 'autoline_exception'
 'partner_name_changed' 'part_availability' 'partner_changed'
 'customer_labor_statement' 'damage' 'workorder_finished'
 'maintenance_rejected' 'diagnosis_rejected' 'key_dropped'
 'customer_contact']


In [ ]:
koppeltabel_df.head()

,workorder_id,work_id
0,172569,1
1,172569,1
2,175844,1
3,175844,1
4,175956,1


In [ ]:
voertuigen_df.head()

,id,car_brand,car_model,coachwork,transmission,car_implementation_id,implementation_code,model_type,model,model_execution_group,...,bruto_bpm,catalog_price,datum_eerste_afgifte_nl,datum_eerste_toelating,description,is_exported,apk_expiration_date,note,created,modified
0,198887,SKOD,FABIA,STATIONWAGEN,A,152.0,NaN,6Y53F3,NaN,NaN,...,NaN,0.0,2003-07-10,2003-07-10,FABIA 1.4 Automaat Elegance 55kW / 75pk A,NaN,2026-05-19,NaN,2021-05-25 20:39:56,2025-04-22 01:30:30
1,199049,SKOD,OCTAVIA,STATIONWAGEN,A,179.0,NaN,1U5273,NaN,NaN,...,NaN,0.0,2001-01-06,2001-01-06,OCTAVIA 1.9 TDI Automaat Comfort 66kW / 90pk A,NaN,2018-01-11,NaN,2021-05-25 20:39:56,2025-02-05 01:34:04
2,199066,SKOD,OCTAVIA,STATIONWAGEN,A,181.0,NaN,1U52N3,NaN,NaN,...,NaN,0.0,2001-01-25,2001-01-25,OCTAVIA 2.0 Automaat Comfort 85kW / 115pk A,NaN,2020-11-16,NaN,2021-05-25 20:39:57,2022-07-15 08:16:04
3,199127,SKOD,FABIA,STATIONWAGEN,A,152.0,NaN,6Y53F3,NaN,NaN,...,NaN,0.0,2001-03-28,2001-03-28,FABIA 1.4 Automaat Elegance 55kW / 75pk A,NaN,2018-09-25,NaN,2021-05-25 20:39:57,2022-07-15 08:16:08
4,199130,SKOD,FABIA,HATCHBACK,A,152.0,NaN,6Y23F3,NaN,NaN,...,NaN,0.0,2001-03-30,2001-03-30,FABIA 1.4 Automaat Elegance 55kW / 75pk A,NaN,2026-03-20,NaN,2021-05-25 20:39:57,2025-03-11 01:30:38


In [ ]:
taken_df.head(20)

,id,cid,description,created,modified
0,1,BARDIAG,Diagnose Occassion,2021-05-18 11:52:00,2025-11-09 01:30:02
1,8,PAF001,Nieuw Verhuur VW Kost.Rijkl.Maken,2021-05-18 11:52:00,2025-11-09 01:30:02
2,9,PAF002,Nieuw Verhuur Audi Kost.Rijkl.Maken,2021-05-18 11:52:00,2025-11-09 01:30:02
3,10,PAF003,Nieuw Verhuur LKW Kost.Rijkl.Maken,2021-05-18 11:52:00,2025-11-09 01:30:02
4,11,PAF004,Nieuw Verhuur VW Kost.Rijkl.Maken,2021-05-18 11:52:00,2025-11-09 01:30:02
5,12,PAF005,Nieuw Verhuur Seat Kost.Rijkl.Maken,2021-05-18 11:52:00,2025-11-09 01:30:02
6,89,PDE001,Eigen Risico Pon Plus Garantie,2021-05-18 11:52:02,2025-11-09 01:30:02
7,90,PDE002,Ruitconcept voorruit 1e ster-reparatie. Let op! Foto's,2021-05-18 11:52:02,2025-11-09 01:30:02
8,91,PDE003,Ruitconcept voorruit tweede ster-reparatie. Let op! Foto's,2021-05-18 11:52:02,2025-11-09 01:30:02
9,92,PLAATWM,Plaat en Spuitwerk,2021-05-18 11:52:02,2025-11-09 01:30:02


# Datasets opschonen (cleanup)

### Berichten opschonen

In [ ]:
berichten_df = berichten_df[berichten_df['remark_type'] == 'customer_contact'];

drop_columns = [
    'crm_partner_id',
    'to_crm_partner_id',
    'read',
    'remark_status',
    'postpone_datetime',
    'crm_partner_completed_id',
    'completed_datetime',
    'crm_partner_in_progress_id',
    'in_progress_datetime',
    'source_id',
    'created',
    'modified',
    'department'
]


berichten_df = berichten_df.drop(
    # Alleen kollommen die nu bestaan in de dataframe verwijderen
    columns=[col for col in drop_columns if col in berichten_df.columns.tolist()]
)

# klant berichten (1821 entries) ⊆ totale berichten (848127 entries); -99.79%
berichten_df.head()


,id,workorder_id,remark_type,remarks,priority,postpone_reason,remark_parent_id,source
804543,1247972,796945,customer_contact,Klant bericht test,medium,NaN,NaN,NaN
804544,1247973,796945,customer_contact,Afbeelding klantportaal test,medium,NaN,NaN,NaN
804585,1248388,794612,customer_contact,"Goedemorgen,\n\nIk heb bij het inchecken aangegeven geen vervangend vervoer nodig te hebben. Bij nader inzien zou ik graag een fiets willen lenen om naar huis te rijden. Is dit alsnog mogelijk?\n\nMvg, Kristian",medium,NaN,NaN,NaN
804633,1248785,798678,customer_contact,Wat is de situzatie op dit moment? Auto is sinds. Dinsdag in onderzoek. Graag wat duidelijkheid,medium,NaN,NaN,NaN
804674,1249080,797158,customer_contact,"Hoi, ik zie dat het voertuig nog steeds staat op 'ingecheckt'. Is dat de status of zit er wat vertraging in de software? Ik vraag het even ivm afspraken eind vd middag en de inschatting of mijn auto dan al klaar is.",medium,NaN,NaN,NaN


### Taken opschonen

In [ ]:
print(taken_df.shape)
taken_df = taken_df.drop_duplicates(subset="description", keep="last")
taken_df.shape


(21217, 5)


(3138, 5)

In [ ]:
triplets_df.shape

(299, 3)

# Aanmaken van een embedding index - SBERT


In [ ]:
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader
import pandas as pd

train_examples = [
    InputExample(texts=[row.anchor, row.positive, row.negative])
    for _, row in triplets_df.iterrows()
]

# Load base model
model = SentenceTransformer("distiluse-base-multilingual-cased-v1")

# DataLoader
train_dataloader = DataLoader(
    train_examples,
    shuffle=True,
    batch_size=16
)

# Triplet Loss (cosine)
train_loss = losses.TripletLoss(
    model=model,
    distance_metric=losses.TripletDistanceMetric.COSINE,
)

# Train
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=2,
    warmup_steps=10,
    show_progress_bar=True
)

# Save trained model
model.save("sbert-triplet-finetuned")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 3


wandb: You chose "Don't visualize my results"


Step,Training Loss


## Load SBERT Model and Generate Embeddings

### Subtask:
Load a pre-trained multilingual SBERT model and generate embeddings for the 'description' column of `taken_df`.


**Reasoning**:
To load the SBERT model and generate embeddings, I need to import `SentenceTransformer`, load the specified pre-trained model, and then create embeddings for the 'description' column of `taken_df`, ensuring to handle any NaN values.



In [ ]:
from sentence_transformers import SentenceTransformer

# Load a pre-trained multilingual SBERT model
# model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
# model = SentenceTransformer('distiluse-base-multilingual-cased-v1')
model = SentenceTransformer("sbert-triplet-finetuned") # using fine my fined tuned model

# Handle NaN values in the 'description' column before generating embeddings
taken_descriptions = taken_df['description'].dropna().tolist()

# Generate embeddings for the descriptions
# embeddings = model.encode(taken_descriptions, show_progress_bar=True)
embeddings = model.encode(
    taken_descriptions,
    show_progress_bar=True,
    convert_to_numpy=True
).astype("float32")

print("SBERT model loaded and embeddings generated.")

The tokenizer you are loading from 'sbert-triplet-finetuned' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.


Batches:   0%|          | 0/99 [00:00<?, ?it/s]

SBERT model loaded and embeddings generated.


In [ ]:
embeddings.shape

(3137, 512)

## Create FAISS Index

### Subtask:
Initialize and populate a FAISS index with the generated embeddings for fast similarity search.


**Reasoning**:
To initialize and populate the FAISS index, I need to import the `faiss` library, determine the dimensionality of the `embeddings`, create an `IndexFlatL2` index with that dimensionality, and then add the `embeddings` to it.



In [ ]:
# import faiss

# # Get the dimensionality of the embeddings
# dimension = embeddings.shape[1]

# # Initialize a FAISS index (IndexFlatL2 for L2 distance)
# index = faiss.IndexFlatL2(dimension) # Euclidean distance

# # Add the embeddings to the index
# index.add(embeddings)

# print(f"FAISS index initialized and populated with {index.ntotal} vectors of dimension {dimension}.")

FAISS index initialized and populated with 21216 vectors of dimension 512.


In [ ]:
# import faiss
# import numpy as np

# # Get the dimensionality of the embeddings
# dimension = embeddings.shape[1]

# # Ensure embeddings are float32 (required by FAISS)
# embeddings = embeddings.astype("float32")

# # Normalize embeddings for cosine similarity
# faiss.normalize_L2(embeddings)

# # Initialize FAISS index for inner product (cosine similarity after normalization)
# index = faiss.IndexFlatIP(dimension)

# # Add the embeddings to the index
# index.add(embeddings)

# print(
#     f"FAISS index initialized with cosine similarity "
#     f"({index.ntotal} vectors, dimension {dimension})."
# )

FAISS index initialized with cosine similarity (21216 vectors, dimension 512).


In [ ]:
import faiss
import numpy as np

# Normalize embeddings for cosine similarity
faiss.normalize_L2(embeddings)

dimension = embeddings.shape[1]

# Cosine similarity via inner product
index = faiss.IndexFlatIP(dimension)
index.add(embeddings)

print(
    f"FAISS index initialized with cosine similarity "
    f"({index.ntotal} vectors, dimension {dimension})."
)


FAISS index initialized with cosine similarity (3137 vectors, dimension 512).


**Reasoning**:
The FAISS index has been initialized and populated. Now, I need to define a function that will take a query (customer remark) and search the FAISS index to find the most similar `taken_df` descriptions. This function will be crucial for linking customer remarks to relevant service tasks.



In [ ]:
def search_faiss(query_text, k=5):
    query_embedding = model.encode([query_text])
    distances, indices = index.search(query_embedding, k)

    results = []
    for i, idx in enumerate(indices[0]):
        original_description = taken_df['description'].iloc[idx] # Assuming taken_df descriptions are in the same order as embeddings
        cid = taken_df['cid'].iloc[idx]
        results.append({
            'description': original_description,
            'cid': cid,
            'distance': distances[0][i]
        })
    return results


search_faiss('Ik heb een klacht')

[{'description': 'Diagnose Occassion',
  'cid': 'BARDIAG',
  'distance': np.float32(0.6866435)},
 {'description': 'Wiellager vervangen (achterzijde)',
  'cid': 'H 4258D7B6A',
  'distance': np.float32(0.67002976)},
 {'description': 'Schadereparatie',
  'cid': 'PON935',
  'distance': np.float32(0.667311)},
 {'description': 'SmartLink bedraad, MIB3 PR9WC',
  'cid': '6VA054830A00001',
  'distance': np.float32(0.6663928)},
 {'description': 'Klacht oorzaak en remedie',
  'cid': 'PON930',
  'distance': np.float32(0.65669894)}]

In [ ]:
import time
def search_faiss(query_text, k=5):
    query_embedding = model.encode([query_text]).astype("float32")
    faiss.normalize_L2(query_embedding)

    scores, indices = index.search(query_embedding, k)

    results = []
    for i, idx in enumerate(indices[0]):
        results.append({
            "description": taken_descriptions[idx],
            "score": float(scores[0][i])
        })
    return results


search_faiss('Kan het Koel- en remvloeistof vervangen worden?')

[{'description': 'Koelmiddellekkagepatroon vervangen',
  'score': 0.8147504329681396},
 {'description': 'Koelvloeistof afgetapt en gevuld',
  'score': 0.812036395072937},
 {'description': 'Koelvloeistof verversen', 'score': 0.8111274242401123},
 {'description': 'Koelmiddelpompriem vervangen', 'score': 0.8102346658706665},
 {'description': 'Remvloeistof verversen', 'score': 0.800450325012207}]